# 04: Point Localization

*Authors: Lars Nitzschke, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup  
3. Algorithms  
    3.1. Slab Decomposition  
    3.2. Vertical Decomposition
4. References  

## 1. Introduction

The **point localization problem** was stated in the lecture as follows: Preprocess a planar subdivision such that for any query point q, the face of the subdivision containing q can be given efficiently.

For the **one dimensional** problem it was shown, that the problem can be solved efficiently using a balanced binary search tree.

[Image with example ?]

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external sources, including our module for generic data structures, our module for geometric primitives and operations as well as our module for visualisation purposes. 
These modules are explained in [notebook no. 00](./00-Basics.ipynb).

In [1]:
# Python standard library imports
import math
from typing import Any, Optional
from itertools import combinations

# Data structure, geometry and visualisation module imports
from modules.data_structures import BinaryTree, BinaryTreeDict, Comparator, ComparisonResult as CR, DoublyConnectedSimplePolygon, HalfEdge, DoublyConnectedEdgeList, PointLocation
from modules.geometry import Point, PointReference, PointSequence, LineSegment, Orientation as ORT, EPSILON, Rectangle
from modules.visualisation import VisualisationTool, PointLocationInstance, DCELInstance, SlabDecompositionMode, VerticalExtensionMode

Additionally, we create an object for our visualisation tool and register a few example instances.

In [2]:
visualisation_tool = VisualisationTool(400, 400, DCELInstance(drawing_epsilon=10))
canvas_size = min(visualisation_tool.width, visualisation_tool.height)

c = 0.5 * canvas_size
r = 0.75 * c
s = c - r
t = c + r
u = (t - s) / 30

points = [Point(s + 20*u, t),        Point(s + 26*u, t -  6*u), Point(s + 30*u, t -  8*u), Point(s + 28*u, t - 10*u),
          Point(s + 18*u, t - 12*u), Point(s + 29*u, s + 14*u), Point(s + 27*u, s +  6*u), Point(s + 22*u, s +  1*u),
          Point(s + 14*u, s +  4*u), Point(s + 10*u, s),        Point(s +  2*u, s +  5*u), Point(s + 12*u, s + 12*u),
          Point(s +  4*u, s + 13*u), Point(s +  1*u, s + 16*u), Point(s +  3*u, t - 13*u), Point(s +  5*u, t -  4*u),
          Point(s + 10*u, t -  1*u), Point(s + 14*u, t - 15*u), Point(s + 18*u, s +  8*u)]
          
edges = [(0,1), (1,2), (2,3), (3,4), (4,5), (5,6), (6,7), (7,8), (8,9), (9,10), (10,11), (11,12), (12,13), (13,14),
         (14,15), (15,16), (16,0), (1,3), (4,16), (4,17), (12,17), (14,17), (5,11), (6,18), (10,18)]

# For adding an inner component:
#points.extend([Point(s + 16*u, t -  6*u), Point(s + 20*u, t -  2*u), Point(s + 22*u, t -  8*u)])
#edges.extend([(19,20), (20, 21), (18,21)])

dcel = DoublyConnectedEdgeList(points, edges)
visualisation_tool.register_example_instance("Course Example", dcel)

visualisation_tool.display()

In [3]:
# visualisation_tool = VisualisationTool(400, 400, PointLocationInstance())
# pl = PointLocation(Rectangle(Point(0, 0), Point(400, 400)), dcel)
# visualisation_tool.register_example_instance("Testing example", pl)

def point_location(dcel: DoublyConnectedEdgeList) -> PointSequence:
    pl = PointLocation(Rectangle(Point(0, 0), Point(400, 400)), dcel)
    point_list: list[PointReference] = []
    for line_segment in pl._vertical_decomposition.line_segments:
        point_list.append(PointReference([line_segment.left], 0))
        point_list.append(PointReference([line_segment.right], 0))

    for trapezoid in pl._vertical_decomposition.trapezoids:
        for point in point_list:
            if point == trapezoid.left_point or point == trapezoid.right_point:
                point_above = Point(point.x, trapezoid.top_line_segment.y_from_x(point.x))
                point_below = Point(point.x, trapezoid.bottom_line_segment.y_from_x(point.x))
                if len(point.container) == 1:
                    point.container.append(point_above)
                elif point.container[1].y < point_above.y:
                    point.container[1] = point_above
                if len(point.container) == 2:
                    point.container.append(point_below)
                elif point.container[2].y > point_below.y:
                    point.container[2] = point_below

    #print(point_list)

    point_seq = PointSequence()
    for point in point_list:
        point_seq.append(point)
    
    return point_seq


visualisation_tool.register_algorithm("Build Vert. Decomp.", point_location, VerticalExtensionMode())

visualisation_tool.display()

# visualisation_tool._example_buttons[0].click()
# visualisation_tool._algorithm_buttons[0].click()
# visualisation_tool._random_button.click()

In [ ]:
# TODO: Drawing and instance order clean up
# TODO: Animation of VD drawings
# TODO: Actual point localization implementation
# TODO: Point localization animation
# TODO: VD: Problem with vertical segements (unallowed neighbors)
# TODO: VD-RIC

# TODO: Slab decompostion
# TODO: Slab decompostion drawings

In [4]:
#print("A")
#visualisation_tool.handle_click_on_multi_canvas(200, 200)
#print("B")
#visualisation_tool.handle_click_on_multi_canvas(200, 200)
#
#
#print("C")
#visualisation_tool.handle_click_on_multi_canvas(100, 200)
#print("D")
#visualisation_tool.handle_click_on_multi_canvas(100, 200)
#
#
#print("E")
#visualisation_tool.handle_click_on_multi_canvas(50, 50)
#print("F")
#visualisation_tool.handle_click_on_multi_canvas(50, 50)
#
#
#print("G")
#visualisation_tool.handle_click_on_multi_canvas(200, 200)

## 3. Algorithms

### 3.1 Slab Decomposition

In [5]:
class FloatComparator(Comparator[float]):
    def compare(self, item: Any, key: float) -> CR:
        if not isinstance(item, float):
            raise TypeError("Only floats can be compared to floats")
        elif abs(item - key) <= EPSILON:
            return CR.MATCH
        elif item - key > EPSILON:
            return CR.BEFORE
        else:
            return CR.AFTER
        
class SD_LineSegementComparator(Comparator[LineSegment]):
    def compare(self, item: Any, key: LineSegment) -> CR:
        if not isinstance(item, LineSegment):
            raise TypeError("Only LineSegments can be compared to LineSegements")
        elif abs(item.lower - key.lower) <= EPSILON:
            return CR.MATCH
        elif item - key > EPSILON:
            return CR.BEFORE
        else:
            return CR.AFTER

In [6]:
def slab_decomposition(dcel: DoublyConnectedSimplePolygon) -> PointSequence:
    return SlabDecomposition(dcel).do()

class SlabDecomposition:
    def __init__(self, dcel: DoublyConnectedSimplePolygon):
        self._x_comparator = FloatComparator()
        self._y_comparator = SD_LineSegementComparator()
        self._data_tree: BinaryTreeDict[float, BinaryTreeDict[LineSegment, Any]] = BinaryTreeDict(self._x_comparator) #TODO: Any -> DCEL-face
        for vertex in dcel.vertices():
            self._data_tree.update(vertex.point.x, BinaryTreeDict(self._y_comparator))
            for vertex in dcel.vertices():
                edge = vertex.edge
                cr: CR = self._comparator.compare(edge.origin.y, edge.next.origin.y)
    def do():
        pass

In [7]:
visualisation_tool.register_algorithm("Slab Decomposition", slab_decomposition, SlabDecompositionMode())

### 3.2 Vertical Decomposition